In [59]:
import numpy as np
import random
import MySQLdb
import scipy.spatial
import pickle
from math import sqrt
import sys

In [60]:
# Database Connection Setup

db = MySQLdb.connect(host="localhost",user="root",passwd="jalaz",db="kitabghar")
cur = db.cursor()

In [61]:
# Hash the UserId into indices for better matrix calculations

hashUsers = {}
query = "SELECT userid from users"
cur.execute(query)

for i,row in enumerate(cur.fetchall()):
    hashUsers[row[0]] = i
users = len(hashUsers)

In [62]:
# Hash the BookId into indices for better matrix calculations

hashBooks = {}
query = "SELECT bookid from books"
cur.execute(query)

for i,row in enumerate(cur.fetchall()):
    hashBooks[row[0]] = i
books = len(hashBooks)

In [63]:
# Grab Dynamic Ratings dircetly from the Database

ratingData = []
query = "SELECT userid,bookid,rating from reviews"
cur.execute(query)

for row in cur.fetchall():
    r = [hashUsers[row[0]], hashBooks[row[1]], int(row[2])]
    ratingData.append(r)

In [64]:
# Grab the total number of Users who have rated

query = "SELECT count(*) from reviews GROUP BY userid"
cur.execute(query)
print(len(cur.fetchall()))

5


In [65]:
# Grab the total number of Books Rated

query = "SELECT count(*) from reviews GROUP BY bookid"
cur.execute(query)
print(len(cur.fetchall()))

128


In [66]:
# Create the Data Matrix

dataMatrix = np.zeros((users,books))
for entry in ratingData:
    dataMatrix[entry[0]][entry[1]] = entry[2]
    
print(dataMatrix)

[[ 4.  4.  0. ...,  4.  0.  0.]
 [ 3.  4.  0. ...,  1.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 5.  2.  2. ...,  0.  0.  2.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [67]:
# Getting Similarity Matrix for our data

item_similarity_cosine = np.zeros((books,books))
for book1 in range(books):
    for book2 in range(books):
        if np.count_nonzero(dataMatrix[:, book1]) and np.count_nonzero(dataMatrix[:, book2]):
            item_similarity_cosine[book1][book2] = 1-scipy.spatial.distance.cosine(dataMatrix[:, book1], dataMatrix[:, book2])
        
    
print(item_similarity_cosine)

[[ 1.          0.6486493   0.61545745 ...,  0.45866432  0.44038551
   0.43519414]
 [ 0.6486493   1.          0.2773501  ...,  0.87028527  0.24806947
   0.58834841]
 [ 0.61545745  0.2773501   1.         ...,  0.          0.          0.70710678]
 ..., 
 [ 0.45866432  0.87028527  0.         ...,  1.          0.26311741
   0.41602515]
 [ 0.44038551  0.24806947  0.         ...,  0.26311741  1.          0.31622777]
 [ 0.43519414  0.58834841  0.70710678 ...,  0.41602515  0.31622777  1.        ]]
